# Google Colab Setup

In [1]:
!git clone https://github.com/haidarihza/sistem-penilaian-kompetensi-v2

!pip install batchbald_redux

import sys
sys.path.append('/content/sistem-penilaian-kompetensi-v2/Penilaian Kompetensi Inggris')

Cloning into 'sistem-penilaian-kompetensi-v2'...
remote: Enumerating objects: 19203, done.
remote: Counting objects: 100% (1075/1075), done.
remote: Compressing objects: 100% (372/372), done.
remote: Total 19203 (delta 675), reused 1075 (delta 675), pack-reused 18128
Receiving objects: 100% (19203/19203), 163.77 MiB | 17.83 MiB/s, done.
Resolving deltas: 100% (2472/2472), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 6.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvi

In [2]:
from google.colab import files
uploaded = files.upload()

Saving dataset.json to dataset.json


In [3]:
import json
for key in uploaded.keys():
  raw_data = json.loads(uploaded[key])['data']

# Evaluation Setup

In [4]:
from src.model import *
import torch

In [5]:
device = torch.device('cuda')

In [6]:
# Set seed
torch.manual_seed(0)

# Active Learning Evaluation

Data split: 7 pool sets / 2 eval sets (1 set = 4 data points)<br>
Base model: cross-encoder/nli-deberta-v3-base<br>

Acquisition function: Random<br>
Acquisition size: 10<br>
Acquisition step: 2<br>

In [7]:
pool_data = raw_data[:7]
eval_data = raw_data[7:]

transcripts = [transcript for d in pool_data for transcript in d['transcripts']]
competence_sets = [d['competence_levels'] for d in pool_data for _ in range(len(d['transcripts']))]
labels = [label for d in pool_data for label in d['labels']]
pool_dataset = TCDataset(transcripts, competence_sets, labels)

transcripts = [transcript for d in eval_data for transcript in d['transcripts']]
competence_sets = [d['competence_levels'] for d in eval_data for _ in range(len(d['transcripts']))]
labels = [label for d in eval_data for label in d['labels']]
eval_dataset = TCDataset(transcripts, competence_sets, labels)

In [8]:
model = CompetenceModel.load('cross-encoder/nli-deberta-v3-base', 'crossencoder', state_dict_path=None, device=device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/417 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/18.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:562: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/738M [00:00<?, ?B/s]

In [9]:
# Model prediction on eval dataset (initial)

with torch.no_grad():
    scores = model(eval_dataset.transcripts, eval_dataset.competence_sets)
    acc = torch.sum(torch.argmax(scores, dim=1) == torch.tensor(eval_dataset.label_indices, device=device)).item() / scores.size(0)

print('Predict probabilities:')
print(scores)
print(f'Accuracy: {acc:.2f}')

Predict probabilities:
tensor([[8.8539e-01, 2.4814e-03, 2.4600e-03, 1.0966e-01],
        [3.1869e-01, 5.6854e-02, 2.3564e-01, 3.8882e-01],
        [9.1472e-02, 7.9833e-02, 2.5287e-01, 5.7583e-01],
        [1.7241e-03, 1.1876e-03, 1.6624e-03, 9.9543e-01],
        [7.4673e-01, 2.5297e-01, 1.4159e-04, 1.6520e-04],
        [3.3627e-03, 9.5070e-01, 4.5272e-02, 6.6608e-04],
        [5.4032e-02, 1.3424e-01, 8.0497e-01, 6.7584e-03],
        [1.4482e-01, 7.0768e-01, 9.1833e-02, 5.5661e-02]], device='cuda:0')
Accuracy: 0.62


## Acquisition Step 1

In [10]:
batch = get_random_batch(len(pool_dataset), batch_size=10)

In [11]:
train_dataset = torch.utils.data.Subset(pool_dataset, batch.indices)

transcripts = [t for i, t in enumerate(pool_dataset.transcripts) if i not in batch.indices]
competence_sets = [c for i, c in enumerate(pool_dataset.competence_sets) if i not in batch.indices]
labels = [l for i, l in enumerate(pool_dataset.label_indices) if i not in batch.indices]
pool_dataset = TCDataset(transcripts, competence_sets, labels)

In [12]:
# Model training on 5 epochs

model = CompetenceModel.load('cross-encoder/nli-deberta-v3-base', 'crossencoder', state_dict_path=None, device=device)

model.fit(train_dataset, eval_dataset, epochs=5, batch_size=4, early_stop=False,
          optimizer_cls=torch.optim.Adam, optimizer_params={'lr': 1e-5})

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:562: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Epoch 1 | Train Loss = 1.8172 | Train Acc = 0.3000 | Val Loss = 1.9835 | Val Acc = 0.6250
Epoch 2 | Train Loss = 1.1043 | Train Acc = 0.5000 | Val Loss = 2.0035 | Val Acc = 0.6250
Epoch 3 | Train Loss = 0.8803 | Train Acc = 0.6000 | Val Loss = 2.0187 | Val Acc = 0.6250
Epoch 4 | Train Loss = 0.8327 | Train Acc = 0.6000 | Val Loss = 1.8968 | Val Acc = 0.7500
Epoch 5 | Train Loss = 0.5015 | Train Acc = 0.9000 | Val Loss = 1.5916 | Val Acc = 0.6250


In [13]:
# Model prediction on eval dataset

with torch.no_grad():
    scores = model(eval_dataset.transcripts, eval_dataset.competence_sets)
    acc = torch.sum(torch.argmax(scores, dim=1) == torch.tensor(eval_dataset.label_indices, device=device)).item() / scores.size(0)

print('Predict probabilities:')
print(scores)
print(f'Accuracy: {acc:.2f}')

Predict probabilities:
tensor([[5.6404e-01, 1.3924e-03, 1.4614e-03, 4.3310e-01],
        [4.0967e-01, 3.5597e-02, 1.8211e-01, 3.7263e-01],
        [2.4128e-03, 1.5747e-03, 9.2865e-01, 6.7359e-02],
        [8.3424e-04, 5.9483e-04, 2.5145e-03, 9.9606e-01],
        [5.0000e-01, 4.9855e-01, 1.0801e-03, 3.6877e-04],
        [2.8031e-01, 3.8182e-01, 3.3701e-01, 8.5557e-04],
        [9.2114e-02, 4.5905e-01, 4.4652e-01, 2.3186e-03],
        [9.5625e-03, 7.4970e-01, 2.3888e-01, 1.8654e-03]], device='cuda:0')
Accuracy: 0.62


## Acquisition Step 2

In [14]:
batch = get_random_batch(len(pool_dataset), batch_size=10)

In [15]:
train_dataset = torch.utils.data.ConcatDataset([train_dataset, torch.utils.data.Subset(pool_dataset, batch.indices)])

transcripts = [t for i, t in enumerate(pool_dataset.transcripts) if i not in batch.indices]
competence_sets = [c for i, c in enumerate(pool_dataset.competence_sets) if i not in batch.indices]
labels = [l for i, l in enumerate(pool_dataset.label_indices) if i not in batch.indices]
pool_dataset = TCDataset(transcripts, competence_sets, labels)

In [16]:
# Model training on 5 epochs

model = CompetenceModel.load('cross-encoder/nli-deberta-v3-base', 'crossencoder', state_dict_path=None, device=device)

model.fit(train_dataset, eval_dataset, epochs=5, batch_size=4, early_stop=False,
          optimizer_cls=torch.optim.Adam, optimizer_params={'lr': 1e-5})

Epoch 1 | Train Loss = 1.9958 | Train Acc = 0.3500 | Val Loss = 0.9391 | Val Acc = 0.5000
Epoch 2 | Train Loss = 1.2869 | Train Acc = 0.4500 | Val Loss = 0.9735 | Val Acc = 0.5000
Epoch 3 | Train Loss = 1.2028 | Train Acc = 0.5500 | Val Loss = 0.9772 | Val Acc = 0.5000
Epoch 4 | Train Loss = 1.2153 | Train Acc = 0.3000 | Val Loss = 0.9644 | Val Acc = 0.5000
Epoch 5 | Train Loss = 0.9852 | Train Acc = 0.6500 | Val Loss = 0.9541 | Val Acc = 0.5000


In [17]:
# Model prediction on eval dataset after training

with torch.no_grad():
    scores = model(eval_dataset.transcripts, eval_dataset.competence_sets)
    acc = torch.sum(torch.argmax(scores, dim=1) == torch.tensor(eval_dataset.label_indices, device=device)).item() / scores.size(0)

print('Predict probabilities:')
print(scores)
print(f'Accuracy: {acc:.2f}')

Predict probabilities:
tensor([[4.7618e-01, 6.3024e-02, 5.4442e-02, 4.0636e-01],
        [3.4024e-01, 1.5144e-01, 2.5078e-01, 2.5754e-01],
        [9.7765e-02, 1.0298e-01, 3.5118e-01, 4.4808e-01],
        [2.1231e-02, 1.9916e-02, 2.6247e-02, 9.3261e-01],
        [9.9496e-01, 3.9360e-03, 4.2447e-04, 6.8384e-04],
        [1.3649e-01, 8.0516e-01, 2.8237e-02, 3.0121e-02],
        [3.3246e-01, 3.3194e-01, 1.4327e-01, 1.9233e-01],
        [2.1315e-01, 4.5723e-01, 1.5093e-01, 1.7869e-01]], device='cuda:0')
Accuracy: 0.50
